In [1]:
import argparse
import simplejson as json
import os
import random
from tqdm import tqdm

seed = 42
home_dir = os.getenv('HOME')
num_traineval = 7000
frac_traineval = num_traineval / 90447

# parser = argparse.ArgumentParser()
# parser.add_argument("--save_folder", default='hotpot-squad-q-subqs-subas', type=str)
# args = parser.parse_args()
# save_folder = args.save_folder
save_folder = 'hotpot-squad-q-predsubqs-predsubas-no-co'
print(save_folder)

with open(home_dir + '/research/data/hotpot-orig/train.json') as f:
    train = json.load(f)

qids = [example['_id'] for example in train]
shuffler = random.Random(seed)
shuffler.shuffle(qids)
assert qids[0] == '5abed9f45542994516f4545a', 'Unexpected QID after seeded random shuffle'
traineval_qids = set(qids[:num_traineval])

hotpot-squad-q-predsubqs-predsubas-no-co


In [2]:
# # One-time: Saving new gold file
# traineval = [example for example in train if example['_id'] in traineval_qids]
# with open('/Users/ethanperez/research/data/hotpot-orig/traineval.json', 'w') as f:
#     json.dump(traineval, f, indent=2)

In [3]:
with open(home_dir + '/research/data/squad/train.json') as f:
    squad = json.load(f)

squad_titles = []
for article in tqdm(squad['data']):
    squad_titles.append(article['title'])

squad_shuffler = random.Random(seed)
squad_shuffler.shuffle(squad_titles)
assert squad_titles[0] == 'Asphalt', 'Unexpected title after seeded random shuffle'
num_traineval_squad_titles = int(round(len(squad_titles) * frac_traineval))
print('num_traineval_squad_titles:', num_traineval_squad_titles)
traineval_squad_titles = set(squad_titles[:num_traineval_squad_titles])

100%|██████████| 442/442 [00:00<00:00, 988737.26it/s]

num_traineval_squad_titles: 34


In [4]:
# One-time: Saving new gold file
squad_traineval = {
    'data': [article for article in squad['data'] if article['title'] in traineval_squad_titles],
    'version': squad['version']
}
with open(home_dir + '/research/data/squad/traineval.json', 'w') as f:
    json.dump(squad_traineval, f, indent=2)

In [5]:
with open(home_dir + '/research/data/' + save_folder + '/train.json') as f:
    data = json.load(f)

In [6]:
assert None not in traineval_qids
new_data = {
    'version': data['version'],
    'data': []
}
for article in tqdm(data['data']):
    if (article['paragraphs'][0].get('_id') in traineval_qids) or \
        (('_id' not in article['paragraphs'][0]) and (article['title'] in traineval_squad_titles)):
        new_data['data'].append(article)
print('num articles', len(new_data['data']))  # should be 7000 (hotpot) + 34 (squad)

100%|██████████| 90889/90889 [00:00<00:00, 719195.59it/s]

num articles 7034


In [7]:
with open(home_dir + '/research/data/' + save_folder + '/traineval.json', 'w') as f:
    json.dump(new_data, f, indent=2)